# Zeitzonen exportieren

In [65]:
from pathlib import Path
import geopandas as gpd
import json
import os
import consts

## Create 15° Lines

In [ ]:
features = []

for i in range(0, 24):

    lon = (i * 15) - 180 + 7.5

    features.append({
        'type': 'Feature',
        'geometry': {
            'type': 'LineString',
            'coordinates': [
                [lon, -90],
                [lon, 90],
            ]
        }
    })


json.dump({
    'type': 'FeatureCollection',
    'features': features
}, open(Path('../export/web/timezone-lines.geojson'), 'w', encoding='utf-8'), ensure_ascii=False)

## Add Timezones to Country shapes

In [35]:
gdf_countries_raw = gpd.read_file(Path('../data/countries-europe.gpkg'))

In [68]:
gdf_countries = gdf_countries_raw.copy()

# Set Timezone
gdf_countries['timezone_is'] = 1
gdf_countries.loc[gdf_countries['ISO_A2_EH'].isin(['PT', 'GB', 'IE', 'IS']), 'timezone_is'] = 0
gdf_countries.loc[gdf_countries['ISO_A2_EH'].isin(['FI', 'EE', 'LT', 'LV', 'RO', 'BG', 'GR', 'MD', 'UA']), 'timezone_is'] = 2
gdf_countries.loc[gdf_countries['ISO_A2_EH'].isin(['RU', 'BY', 'TK']), 'timezone_is'] = 3

# Timezone should
gdf_countries.loc[gdf_countries['ISO_A2_EH'].isin(['IS', 'IE', 'PT']), 'timezone_should'] = -1
gdf_countries.loc[gdf_countries['ISO_A2_EH'].isin(['ES', 'GB', 'FR', 'BE', 'NL', 'LU']), 'timezone_should'] = 0
gdf_countries.loc[gdf_countries['ISO_A2_EH'].isin(['DE', 'CH', 'IT', 'AT', 'PL', 'CZ', 'SI', 'HR', 'BA', 'ME', 'AL', 'MK', 'XK', 'RS', 'HU', 'SK', 'NO', 'SE', 'DK']), 'timezone_should'] = 1
gdf_countries.loc[gdf_countries['ISO_A2_EH'].isin(['FI', 'EE', 'LT', 'LV', 'UA', 'RO', 'BG', 'GR', 'MD', 'BY']), 'timezone_should'] = 2

# Drop Countries
gdf_countries = gdf_countries[~gdf_countries['ISO_A2_EH'].isin(['RU', 'TK'])]

gdf_countries = gdf_countries[['geometry', 'ISO_A2_EH', 'timezone_is', 'timezone_should']]

gdf_countries.to_file(Path('../export/web/countries.geojson'))

In [69]:
# Run Tippecanoe
os.system(f"tippecanoe -zg -o {Path('../export/web/countries.mbtiles')} --drop-densest-as-needed {Path('../export/web/countries.geojson')} --force")

For layer 0, using name "countries"
50 features, 581724 bytes of geometry, 263 bytes of string pool
Choosing a maxzoom of -z0 for features typically 2113883 feet (644312 meters) apart, and at least 404454 feet (123278 meters) apart
Choosing a maxzoom of -z5 for resolution of about 13965 feet (4256 meters) within features
  99.9%  5/14/13  


0